In [ ]:
#Imports
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from import_dfs import readcsvs
from sktime.datatypes import convert_to
from sktime.datatypes import MTYPE_REGISTER
from sktime.datatypes import SCITYPE_REGISTER

from get_all_metrics_with_tags import get_all_metrics_with_tags
better_file_list, y = get_all_metrics_with_tags(r"F:\Master\Kubernetes\sockshop\microservices-demo\query\automated\generated_csvs_4")

df = readcsvs(better_file_list, remove_monotonic_increasing=True)
converted = convert_to(df, 'df-list')
df
#Get a large enough data set, and eventually every single metric is cleansed from somewhere. When then removing unique columns, you end up with zero, having cleaned away all of you data.
#Potential solves: Perform PCA on each dataset before doing any cleaning at all. This boils down the amount of metrics measured to an approximation of the most important ones.
#Other potential solve: Do not remove unique columns, and instead use only datatypes and classifiers that can handle unique features. This sounds risky as it will still need equal features to compare somewhere.
#Other potential solve: Changing the metrics that sometimes monotonically increase, to instead work on a differential level. Check if there is a built in method for that perhaps?


In [ ]:
#Creating DF. Metod: PCA
df = readcsvs(better_file_list, remove_monotonic_increasing=True, PCA=True)
converted = convert_to(df, 'df-list')
df

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
pca = PCA(n_components=25)
principal_comoponents = pca.fit_transform(scaled_data)
newdf = pd.DataFrame(principal_comoponents, index=df.index)
newdf


In [ ]:
#Rocket,  removed monotonic, removed monotonically increasing, standardscaled

from sktime.classification.kernel_based import RocketClassifier
from sklearn.model_selection import train_test_split


X_train,X_test, y_train, y_test = train_test_split(convert_to(df, 'df-list'), y)

rocket = RocketClassifier()
fitted = rocket.fit(X_train, np.array(y_train))
print(fitted.predict(X_test))
print(fitted.score(X_test, np.array(y_test)))


In [ ]:
#Rocket with RobustScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import RidgeClassifierCV
robscale = RobustScaler()
X_train_multiindex = convert_to(X_train, 'pd-multiindex')
robscale.fit(X_train_multiindex)
X_train_robust = robscale.transform(X_train_multiindex)
X_test_robust = robscale.transform(convert_to(X_test, 'pd-multiindex'))

classifier = RidgeClassifierCV(alphas=np.logspace(-3,3,10))
print(X_train_multiindex.shape)
X_train_robust = np.reshape(X_train_robust, X_train_multiindex.shape)
classifier.fit(X_train_robust, y_train)